In [1]:
import scanpy
import hdf5plugin
import pandas
import numpy
import anndata
import scipy

In [2]:
base_names = [
    "MEF",
    "Pool1_mESC",
    "Pool2_mESC",
    "Repro_Day2_BFP",
    "Repro_Day2_Hic2",
    "Repro_Day4_BFP",
    "Repro_Day4_Hic2",
    "Repro_Day6_BFP",
    "Repro_Day6_Hic2",
    "Repro_Day9_BFP",
    "Repro_Day9_Hic2",
    "Repro_Day12_BFP",
    "Repro_Day12_Hic2",
]

seurat_batches = [
    'MEF',
    'Pool1_mESC',
    'Pool2_mESC',
    'Repro_Day2_BFP',
    'Repro_Day2_Hic2',
    'Repro_Day4_BFP',
    'Repro_Day4_Hic2',
    'Repro_Day6_BFP',
    'Repro_Day6_Hic2',
    'Repro_Day9_BFP',
    'Repro_Day9_Hic2',
    'Repro_Day12_BFP',
    'Repro_Day12_Hic2',
]


groups = [
    "MEF",
    "mESC",
    "mESC",
    "control",
    "Hic2",
    "control",
    "Hic2",
    "control",
    "Hic2",
    "control",
    "Hic2",
    "control",
    "Hic2",
]

days = [
    0,
    0,
    0,
    2,
    2,
    4,
    4,
    6,
    6,
    9,
    9,
    12,
    12,
]

samples = [
    "MEF",
    "mESC",
    "mESC",
    "Day 2 Control",
    "Day 2 Hic2 OX",
    "Day 4 Control",
    "Day 4 Hic2 OX",
    "Day 6 Control",
    "Day 6 Hic2 OX",
    "Day 9 Control",
    "Day 9 Hic2 OX",
    "Day 12 Control",
    "Day 12 Hic2 OX"
]

sample_order = [
    "MEF",
    "Day 2 Control",
    "Day 4 Control",
    "Day 6 Control",
    "Day 9 Control",
    "Day 12 Control",
    "Day 2 Hic2 OX",
    "Day 4 Hic2 OX",
    "Day 6 Hic2 OX",
    "Day 9 Hic2 OX",
    "Day 12 Hic2 OX",
    "mESC",
]

sample_colors = [
    '#cbc9ca',
    "#ffd1be", 
    '#f3ad91',
    '#eb684b',
    '#da2e24',
    '#a81918',
    "#ced8f0",
    '#b6d8e5',
    '#71add9',
    '#3180bd',
    '#164f9b',
    '#0a652c'
]

In [3]:
# Read Seurat object to add UMAP and FDL
adata_seurat = scanpy.read_h5ad("/scratch/lema/m26_losu/seurat_object.h5ad")
fdl = pandas.read_csv("/home/avesta/m26_losu/Hic2/fdl.csv", index_col=0)
adata_seurat.obsm['X_fdl'] = numpy.array(fdl)

In [4]:
# Function to convert mtx to anndata

def mtx_to_anndata(path, filter_cells=True):
    """Generate an anndata object from the STAR aligner output folder.
    Based on https://github.com/alexdobin/STAR/issues/774#issuecomment-850477636."""
    path=path
    # Load Read Counts
    X = scanpy.read_mtx(path+'Velocyto/raw/spliced.mtx')

    # Transpose counts matrix to have Cells as rows and Genes as cols as expected by AnnData objects
    X = X.X.transpose()

    # Load the 3 matrices containing Spliced, Unspliced and Ambigous reads
    mtxU = numpy.loadtxt(path+'Velocyto/raw/unspliced.mtx', skiprows=3, delimiter=' ')
    mtxS = numpy.loadtxt(path+'Velocyto/raw/spliced.mtx', skiprows=3, delimiter=' ')
    mtxA = numpy.loadtxt(path+'Velocyto/raw/ambiguous.mtx', skiprows=3, delimiter=' ')
    mtxG = numpy.loadtxt(path+'Gene/raw/matrix.mtx', skiprows=3, delimiter=' ')

    # Extract sparse matrix shape informations from the third row
    shapeU = numpy.loadtxt(path+'Velocyto/raw/unspliced.mtx', skiprows=2, max_rows = 1 ,delimiter=' ')[0:2].astype(int)
    shapeS = numpy.loadtxt(path+'Velocyto/raw/spliced.mtx', skiprows=2, max_rows = 1 ,delimiter=' ')[0:2].astype(int)
    shapeA = numpy.loadtxt(path+'Velocyto/raw/ambiguous.mtx', skiprows=2, max_rows = 1 ,delimiter=' ')[0:2].astype(int)
    shapeG = numpy.loadtxt(path+'Gene/raw/matrix.mtx', skiprows=2, max_rows = 1 ,delimiter=' ')[0:2].astype(int)

    # Read the sparse matrix with csr_matrix((data, (row_ind, col_ind)), shape=(M, N))
    # Subract -1 to rows and cols index because csr_matrix expects a 0 based index
    # Traspose counts matrix to have Cells as rows and Genes as cols as expected by AnnData objects

    spliced = scipy.sparse.csr_matrix((mtxS[:,2], (mtxS[:,0]-1, mtxS[:,1]-1)), shape = shapeS).transpose()
    unspliced = scipy.sparse.csr_matrix((mtxU[:,2], (mtxU[:,0]-1, mtxU[:,1]-1)), shape = shapeU).transpose()
    ambiguous = scipy.sparse.csr_matrix((mtxA[:,2], (mtxA[:,0]-1, mtxA[:,1]-1)), shape = shapeA).transpose()
    any = scipy.sparse.csr_matrix((mtxG[:,2], (mtxG[:,0]-1, mtxG[:,1]-1)), shape = shapeG).transpose()

    # Load Genes and Cells identifiers
    obs = pandas.read_csv(path+'Velocyto/raw/barcodes.tsv',
                  header = None, index_col = 0)

    # Remove index column name to make it compliant with the anndata format
    obs.index.name = None
    obs["barcode"] = obs.index

    var = pandas.read_csv(path+'Velocyto/raw/features.tsv', sep='\t', usecols=[0, 1], header=None,
                                    names = ["gene_ids", "gene_names"], index_col = 0)
  
    # Build AnnData object to be used with ScanPy and ScVelo
    adata = anndata.AnnData(X = X, obs = obs, var = var,
                                                 layers = {'spliced': spliced, 'unspliced': unspliced,
                                                           'ambiguous': ambiguous, "matrix": any})
    adata.var_names_make_unique()

    # Subset Cells based on STAR filtering
    if filter_cells:
        selected_barcodes = pandas.read_csv(path+'Gene/filtered/barcodes.tsv', header = None)
        adata = adata[selected_barcodes[0]]

    return adata.copy()

In [ ]:
# Read and combined mtx files

adata_files = []
directory_path = "/scratch/lema/m26_losu/star_counts_gencode"

for index, base_name in enumerate(base_names):
    star_path = f"{directory_path}/{base_name}/"
    group = groups[index]
    day = days[index]
    print(f"Loading file: {base_name}")
    
    # Read loom file
    adata_star = mtx_to_anndata(star_path, filter_cells=False)
    adata_star.obs["batch"] = base_name
    adata_star.obs["sample"] = samples[index]
    adata_star.obs["group"] = groups[index]
    adata_star.obs["day"] = days[index]
    adata_star.var_names_make_unique()

    # Add barcode variable
    adata_star.obs["barcode"] = adata_star.obs.index
    adata_star.strings_to_categoricals()

    # Add UMAP and FDL from Seurat object
    seurat_batch = seurat_batches[index]
    sample_seurat = adata_seurat[adata_seurat.obs["sample"]==seurat_batch]
    barcodes_seurat = set(sample_seurat.obs["barcode"])

    # Filter out barcodes not in Seurat object
    adata_star = adata_star[adata_star.obs["barcode"].isin(barcodes_seurat)]

    # Add umap from Seurat object
    umap_map = pandas.DataFrame(
        sample_seurat.obsm['X_umap'], 
        index=sample_seurat.obs['barcode']
    )
    adata_star.obsm['X_umap'] = umap_map.loc[adata_star.obs["barcode"]].values

    # Add FDL
    fdl_map = pandas.DataFrame(
        sample_seurat.obsm['X_fdl'], 
        index=sample_seurat.obs['barcode']
    )
    adata_star.obsm['X_fdl'] = fdl_map.loc[adata_star.obs["barcode"]].values

    adata_files.append(adata_star)

# Concatenate Anndata objects

adata = adata_files[0].concatenate(
    adata_files[1:],
    join="outer",
    batch_key="batch",
    batch_categories=base_names,
    index_unique="-"
)

# Assign colors to samples
adata.obs["sample"] = adata.obs["sample"].cat.reorder_categories(sample_order)
adata.uns["sample_colors"] = sample_colors

Loading file: MEF


/tmp/ipykernel_2288664/2761841717.py:37: ImplicitModificationWarning: Setting element `.obsm['X_umap']` of view, initializing view as actual.
  adata_star.obsm['X_umap'] = umap_map.loc[adata_star.obs["barcode"]].values


Loading file: Pool1_mESC


/tmp/ipykernel_2288664/2761841717.py:37: ImplicitModificationWarning: Setting element `.obsm['X_umap']` of view, initializing view as actual.
  adata_star.obsm['X_umap'] = umap_map.loc[adata_star.obs["barcode"]].values


Loading file: Pool2_mESC


...

/tmp/ipykernel_2288664/2761841717.py:37: ImplicitModificationWarning: Setting element `.obsm['X_umap']` of view, initializing view as actual.
  adata_star.obsm['X_umap'] = umap_map.loc[adata_star.obs["barcode"]].values


Loading file: Repro_Day12_BFP


/tmp/ipykernel_2288664/2761841717.py:37: ImplicitModificationWarning: Setting element `.obsm['X_umap']` of view, initializing view as actual.
  adata_star.obsm['X_umap'] = umap_map.loc[adata_star.obs["barcode"]].values


Loading file: Repro_Day12_Hic2


/tmp/ipykernel_2288664/2761841717.py:37: ImplicitModificationWarning: Setting element `.obsm['X_umap']` of view, initializing view as actual.
  adata_star.obsm['X_umap'] = umap_map.loc[adata_star.obs["barcode"]].values
/home/avesta/m26_losu/miniconda3/envs/velocity/lib/python3.12/site-packages/anndata/_core/anndata.py:1774: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [17]:
print(adata)

AnnData object with n_obs × n_vars = 13315 × 78334
    obs: 'barcode', 'batch', 'sample', 'group', 'day'
    var: 'gene_names'
    uns: 'sample_colors'
    obsm: 'X_umap', 'X_fdl'
    layers: 'spliced', 'unspliced', 'ambiguous', 'matrix'


In [19]:
# Choose gene names as variable names

adata.var["ensemble_ids"] = adata.var_names
adata.var_names = adata.var["gene_names"].astype(str)
if not adata.var_names.is_unique:
    # Make duplicates unique by adding suffix
    adata.var_names_make_unique()
# Rename gene name
adata.var['gene_symbol'] = adata.var['gene_names']
adata.var = adata.var.drop(columns='gene_names')

In [20]:
# Make observation names unique
adata.obs_names_make_unique()

In [21]:
print(adata)

AnnData object with n_obs × n_vars = 13315 × 78334
    obs: 'barcode', 'batch', 'sample', 'group', 'day'
    var: 'ensemble_ids', 'gene_symbol'
    uns: 'sample_colors'
    obsm: 'X_umap', 'X_fdl'
    layers: 'spliced', 'unspliced', 'ambiguous', 'matrix'


In [22]:
adata.write_h5ad(
    "/scratch/lema/m26_losu/splice_counts_star_fdl.h5ad",
    compression=hdf5plugin.FILTERS["zstd"]
)